# Q1 — Fan Vote Inference (主线)

目的：理解如何用 **已知淘汰结果 + 评委分数** 反推每周粉丝投票强度，并输出不确定性。

本 notebook 对应主线代码：
- `src/mcm2026/pipelines/mcm2026c_q0_build_weekly_panel.py`（Q0：把官方 wide 表变成 weekly panel）
- `src/mcm2026/pipelines/mcm2026c_q1_smc_fan_vote.py`（Q1：粉丝票推断 + 不确定性摘要）
- `src/mcm2026/visualizations/q1_visualizations.py`（Q1 画图）

输出文件：
- `outputs/predictions/mcm2026c_q1_fan_vote_posterior_summary.csv`
- `outputs/tables/mcm2026c_q1_uncertainty_summary.csv`

运行建议（在仓库根目录）：
```bash
uv sync
uv run python run_all.py
```


## 先用一句话说清楚 Q1（口语版）

DWTS 每周的淘汰结果，像一个“只公布结果、不公布投票细节”的黑箱。
我们已知：
- 评委分数（公开）
- 谁在这一周被淘汰/离开（公开）
但我们不知道：
- 观众到底投了多少票给谁（隐藏）

Q1 做的事就是：
> **把“观众投票”当成隐藏变量，用“淘汰结果 + 评委分数”的约束去反推它。**

### Q1 的输出你怎么给论文手解释？
- `fan_share_mean`：我们推断的“这一周观众支持度（份额）”的平均值
- `fan_share_p05/p95`：不确定性区间（可以理解成“比较保守的范围”）
- `ess_ratio`：有效样本量占比（越低说明这一周信息不足，结论更不稳）
- `evidence`：这一周淘汰约束强不强（越低说明“很多种粉丝分布都能解释结果”）

一句很实用的写法：
> 我们不仅给出点估计，还用区间与证据强度来标注哪些周可靠、哪些周需要谨慎解释。

In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import pandas as pd

def _find_repo_root(start: Path | None = None) -> Path:
    here = Path.cwd() if start is None else Path(start)
    for p in [here, *here.parents]:
        if (p / 'pyproject.toml').exists():
            return p
    return here

REPO_ROOT = _find_repo_root()
SRC_DIR = REPO_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

REPO_ROOT


In [ ]:
# 工具函数：在 notebook 里自动展示 outputs/figures 下的图片（优先 TIFF，其次提示 EPS 路径）
from pathlib import Path


def show_saved_figures(question: str, stems: list[str]) -> None:
    """Display figures saved by visualization scripts.

    - Prefer TIFF (good for notebook preview)
    - EPS usually cannot be previewed in Jupyter without额外系统依赖，所以这里只打印路径
    """

    try:
        from IPython.display import Image, display
    except Exception:
        print('⚠️ IPython not available; cannot display images inline.')
        return

    q = question.strip().lower()
    tiff_dir = Path(REPO_ROOT) / 'outputs' / 'figures' / q / 'tiff'
    eps_dir = Path(REPO_ROOT) / 'outputs' / 'figures' / q / 'eps'

    shown = 0
    for stem in stems:
        tiff_fp = tiff_dir / f'{stem}.tiff'
        eps_fp = eps_dir / f'{stem}.eps'

        if tiff_fp.exists():
            display(Image(filename=str(tiff_fp)))
            shown += 1
        elif eps_fp.exists():
            print('EPS figure exists (not previewed inline):', eps_fp)

    if shown == 0:
        print('⚠️ No TIFF figures found under:', tiff_dir)


# 用法示例（Q1）：
# show_saved_figures('q1', ['q1_uncertainty_heatmap', 'q1_fan_share_intervals'])

## 一键运行（推荐给新队友/论文手）

如果你只想“点一下就把 Q1 跑完 + 出图 + 在 notebook 里看到图”，运行下面这一格即可。

它会自动补齐依赖：
- 先跑 Q0（生成 processed 数据）
- 再跑 Q1（推断粉丝票 + 输出不确定性表）
- 再调用 Q1 可视化脚本（写出 TIFF/EPS）
- 最后把关键 TIFF 图直接显示在 notebook 里

如果中途报错，会提示你最常见的原因（比如没装依赖/路径不对/outputs 不存在）。

In [ ]:
import traceback

try:
    from mcm2026.pipelines import mcm2026c_q0_build_weekly_panel, mcm2026c_q1_smc_fan_vote
    from mcm2026.visualizations.config import VisualizationConfig, create_output_directories
    from mcm2026.visualizations.q1_visualizations import generate_all_q1_visualizations

    _ = mcm2026c_q0_build_weekly_panel.run()
    q1_out = mcm2026c_q1_smc_fan_vote.run()
    print('✅ Q1 done:', q1_out)

    post = pd.read_csv(q1_out.posterior_summary_csv)
    unc = pd.read_csv(q1_out.uncertainty_summary_csv)
    print('posterior rows:', len(post), 'uncertainty rows:', len(unc))

    viz_cfg = VisualizationConfig()
    out_dirs = create_output_directories(REPO_ROOT / 'outputs' / 'figures', ['Q1'])
    generate_all_q1_visualizations(REPO_ROOT, out_dirs['Q1'], viz_cfg, showcase=False)

    # 在 notebook 里直接展示 TIFF（如果存在）
    from IPython.display import Image, display

    tiff_dir = REPO_ROOT / 'outputs' / 'figures' / 'q1' / 'tiff'
    candidates = [
        tiff_dir / 'q1_uncertainty_heatmap.tiff',
        tiff_dir / 'q1_fan_share_intervals.tiff',
        tiff_dir / 'q1_mechanism_comparison.tiff',
    ]

    shown = 0
    for fp in candidates:
        if fp.exists():
            display(Image(filename=str(fp)))
            shown += 1

    if shown == 0:
        print('⚠️ No TIFF figures found to display yet. They should be under:', tiff_dir)

except ModuleNotFoundError as e:
    print('❌ Import failed. Usually this means you did not add repo/src to sys.path or you are not running in the repo root.')
    print('Error:', e)
except FileNotFoundError as e:
    print('❌ File not found. Usually this means upstream outputs were not generated yet.')
    print('Error:', e)
except Exception as e:
    print('❌ Unexpected error:', e)
    traceback.print_exc()

## 1) （可选）先生成 Q0 处理后的数据

Q1 依赖 Q0 的 processed 数据：
- `data/processed/dwts_weekly_panel.csv`
- `data/processed/dwts_season_features.csv`

如果你已经跑过 `run_all.py`，这里可以跳过。

In [ ]:
from mcm2026.pipelines import mcm2026c_q0_build_weekly_panel

q0_out = mcm2026c_q0_build_weekly_panel.run()
q0_out


## 2) 运行 Q1：粉丝票推断 + 不确定性摘要

Q1 的核心输出是两张表：
- Posterior summary：每周每位选手的 `fan_share_mean/p05/p95` 等
- Uncertainty summary：每周的 `ess_ratio/evidence` 等（用于解释哪些周约束弱/更不确定）

这些不确定性指标会在论文里用于解释：哪些结果可信、哪些周由于信息不足而应谨慎。

In [ ]:
from mcm2026.pipelines import mcm2026c_q1_smc_fan_vote

q1_out = mcm2026c_q1_smc_fan_vote.run()
q1_out


In [ ]:
post = pd.read_csv(q1_out.posterior_summary_csv)
unc = pd.read_csv(q1_out.uncertainty_summary_csv)

post.head(5), unc.head(5)


## 3) 快速检查：不确定性分布

解释口径：
- `ess_ratio` 越低，代表“有效样本量占比”越低，推断越不稳定
- `evidence` 越低，代表淘汰约束/信号更弱（更可能出现“多种粉丝分布都说得通”）

In [ ]:
unc_percent = unc[unc['mechanism'].astype(str) == 'percent'].copy()
unc_percent[['ess_ratio', 'evidence']].describe()


## 4) 插入一段“看得见的例子”：高不确定周 vs 低不确定周

只看 `describe()` 有点抽象。

我们下面做两件非常直观的事：
1) 画出 `evidence` 和 `ess_ratio` 的分布，直观看“哪类周信息弱”。
2) 随机挑一个 **evidence 最低** 的周、一个 **evidence 最高** 的周，
   把当周 top contestants 的 `fan_share_mean` 以及区间（p05/p95）画出来。

你会看到：
- 高不确定周：误差棒更长、名次更不稳定
- 低不确定周：误差棒更短、排序更“硬”

## 常见问答（FAQ）

### `ess_ratio` 是什么？为什么重要？
你可以把它理解成“这周推断靠不靠谱”的一个实用指标。

- **越接近 1**：有效样本量占比高，说明权重不那么极端，推断更稳定
- **越接近 0**：说明只有极少数样本在起作用，这周的结果更不稳

论文口径：
> 我们用 ESS ratio 标注信息不足的周，并在解释这些周的结论时更谨慎。

### `evidence` 是什么？和 `ess_ratio` 有啥区别？
- `evidence` 更像“这一周淘汰约束强不强/信息量大不大”
- `ess_ratio` 更像“在这些约束下，我们的样本权重是不是集中在少数点上”

这两个可以一起用：
- `evidence` 低 + `ess_ratio` 低：最需要谨慎

### 为什么不输出 p-value？
因为 Q1 不是标准的“回归显著性检验”问题。
我们在做的是 **带约束的反推**（推断隐藏变量），所以更合适的表达是：
- 点估计（mean）
- 不确定性区间（p05/p95）
- 信息强度/稳定性指标（evidence/ess_ratio）

### 为什么同时跑 `percent` 和 `rank` 两种机制？
这是为了覆盖题目里常见的两种计票口径：
- `percent`：按份额相加
- `rank`：按排名相加

后面 Q2/Q4 可以基于这两种口径做对比与稳健性论证。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

unc_p = unc[unc['mechanism'].astype(str) == 'percent'].copy()
post_p = post[post['mechanism'].astype(str) == 'percent'].copy()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(pd.to_numeric(unc_p['evidence'], errors='coerce').dropna(), bins=30, color='steelblue', alpha=0.85)
axes[0].set_title('Evidence distribution (percent)')
axes[0].set_xlabel('evidence')
axes[0].set_ylabel('count')
axes[0].grid(True, alpha=0.3)

axes[1].hist(pd.to_numeric(unc_p['ess_ratio'], errors='coerce').dropna(), bins=30, color='darkgreen', alpha=0.85)
axes[1].set_title('ESS ratio distribution (percent)')
axes[1].set_xlabel('ess_ratio')
axes[1].set_ylabel('count')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 选一个“最不确定”的周（evidence 最低）和一个“最确定”的周（evidence 最高）
cand = unc_p.copy()
if 'n_exit' in cand.columns:
    cand = cand[pd.to_numeric(cand['n_exit'], errors='coerce').fillna(0) > 0].copy()

cand = cand.dropna(subset=['season', 'week']).copy()
cand['evidence_num'] = pd.to_numeric(cand['evidence'], errors='coerce')
cand = cand.dropna(subset=['evidence_num']).sort_values('evidence_num')

low = cand.iloc[0]
high = cand.iloc[-1]

print('Low-evidence example:', {'season': int(low['season']), 'week': int(low['week']), 'evidence': float(low['evidence_num'])})
print('High-evidence example:', {'season': int(high['season']), 'week': int(high['week']), 'evidence': float(high['evidence_num'])})

def _plot_week(row, ax, title: str):
    s = int(row['season'])
    w = int(row['week'])
    d = post_p[(post_p['season'] == s) & (post_p['week'] == w)].copy()

    # 如果列不存在就直接返回（避免 notebook 直接崩）
    required = {'fan_share_mean', 'fan_share_p05', 'fan_share_p95', 'celebrity_name'}
    if not required.issubset(set(d.columns)):
        ax.text(0.5, 0.5, f'Missing columns: {sorted(required - set(d.columns))}', ha='center', va='center')
        ax.axis('off')
        return

    d['fan_share_mean'] = pd.to_numeric(d['fan_share_mean'], errors='coerce')
    d['fan_share_p05'] = pd.to_numeric(d['fan_share_p05'], errors='coerce')
    d['fan_share_p95'] = pd.to_numeric(d['fan_share_p95'], errors='coerce')
    d = d.dropna(subset=['fan_share_mean', 'fan_share_p05', 'fan_share_p95']).copy()

    d = d.sort_values('fan_share_mean', ascending=False).head(10)

    x = np.arange(len(d))
    y = d['fan_share_mean'].to_numpy()
    lo = y - d['fan_share_p05'].to_numpy()
    hi = d['fan_share_p95'].to_numpy() - y

    ax.errorbar(x, y, yerr=[lo, hi], fmt='o', capsize=3, color='black', ecolor='gray', alpha=0.85)
    ax.set_xticks(x)
    ax.set_xticklabels([str(n)[:10] for n in d['celebrity_name'].astype(str)], rotation=45, ha='right')
    ax.set_ylabel('fan_share_mean (with p05/p95)')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
_plot_week(low, axes[0], f'Low evidence: S{int(low["season"])} W{int(low["week"])} (evidence={float(low["evidence_num"]):.3f})')
_plot_week(high, axes[1], f'High evidence: S{int(high["season"])} W{int(high["week"])} (evidence={float(high["evidence_num"]):.3f})')
plt.tight_layout()
plt.show()

## 4) 生成 Q1 图

图的输出目录：`outputs/figures/Q1/`（TIFF + EPS）。

In [ ]:
from mcm2026.visualizations.config import VisualizationConfig, create_output_directories
from mcm2026.visualizations.q1_visualizations import generate_all_q1_visualizations

viz_cfg = VisualizationConfig()
out_dirs = create_output_directories(REPO_ROOT / 'outputs' / 'figures', ['Q1'])
generate_all_q1_visualizations(REPO_ROOT, out_dirs['Q1'], viz_cfg, showcase=False)
out_dirs['Q1']
